# EE 467 Homework 1: Twitter Spam Detection

Welcome to the first homework of EE 467! Here we will apply the ML pipline introduced in the second lab to **Twitter spam message detection problems**. In the past few years, social medias like Twitter has become the target of spam posting. Automatic spam posts can not only affect user experience, but they can also spread misinformation and subtly affect public opinions.

In this homework, we will work on a mini Twitter message dataset extracted from the CRESCI-2017 dataset. This dataset contains 50000 Twitter messages, half of which are sent by real Twitter users and half of which generated by spam bots. Your task is to train classifiers that can tell these two kinds of messages apart. This time, you'll go deeper and try multiple feature extractors and classifiers. Like the labs, you will need to fill out blanks in code cells. You are also going to answer a few questions and **optionally** work on improvements to existing pipeline completely on your own.

## Before You Start

1. You can discuss on Slack if you have questions and want to seek help; however, please try your best to **limit the scope of your question** and **avoid asking directly for answers**. You should also **avoid copy-pasting answers and code** from others.
2. You **may use AI assistants** (e.g., ChatGPT, Gemini, etc.) to support your work. If you do, you must **clearly acknowledge** this by stating:

- **The name of the AI assistant**, and  
- **How it was used** (e.g., brainstorming, clarifying concepts, debugging, editing, generating examples)

You **must not copy and paste** AI-generated content directly into your submission without **fully understanding** what it does and being able to **explain it in your own words**.

3. You can **optionally work on this homework with one other student of this course as a team**, but **each of you needs to submit the homework individually**. If you choose to form a group, please include the name of your teammate here: _(Name of your teammate)_.

## Feature Extraction

We will begin our ML pipeline with feature extraction in this homework, as the dataset is good enough and does not need any special pre-processing. As usual, we will load the dataset into memory, extract samples features into vectors and split the dataset into training and test sets.

In [25]:
import pandas as pd

## [ TODO ]
# 1) Load all data from `twitter-mini.csv` into variable `data`
# 2) Preview first few samples
!tar -xf twitter-mini.tar.xz
data = pd.read_csv('twitter-mini.csv')
display(data.head())

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.macl'


,text,label
0,@baydiangirl it definitely moved some church f...,0
1,@suebob Looks like #TXlege made up new rules f...,0
2,RT @saleemchat: #WJChat #A1B 5 yrs: People ski...,0
3,@SajidaBalouch Huhhhhh,0
4,@Dukester_94 im the lad of all bibles,0


In lab 2 we have learned how to use the `CountVectorizer` feature extractor, which counts the occurance of each token (letter here) in a sample (Twitter message text here). This time, we will also try `TfidfVectorizer` and see which feature extractor performs better. Here we will introduce the algorithm it implemented, **TF-IDF**:

> **TF-IDF** is the abbreviation for **term frequency–inverse document frequency**, and is defined as the product of the two frequency. The most classical version of TF-IDF is:
>
> $$
tf-idf(t) = tf(t, d) \cdot idf(t, D) = \frac{f_{t,d}}{|d|} \log \frac{|D|}{|D_t|}
$$
>
> Where $t$ is the term, $D$ is the collection of all documents, $d$ is one of the document that contains $t$ and $D_t$ is the subset of documents that contains term $t$. The advantage of TF-IDF is that frequent and useless words, such as "the", "a" and "an" will receive lower weight, since they appear in almost all documents. On the other hand, unique yet frequent (in any $d \in D_t$) words will get higher weight. Research has shown that TF-IDF is easy to implement and performs relatively well.

Before we run these feature extractors on our dataset, we need to convert each message into a sequence of terms. Since Twitter messages often contain emojis, abbreviations, hashtags and other composition of characters, we can't use the word-level analyzer as we have done in lab 2. Instead, we will generate **character N-grams** from these messages.

> A **character N-gram** consists of N consecutive characters. Here we show all character 1, 2, 3 and 4-grams for word "cold":
> <img src="attachment:char-ngram.jpg" width="50%" />

Both `CountVectorizer` and `TfidfVectorizer` provide support for character N-grams. Refer to `sklearn`'s documents to build a character N-gram analyzer.

In [26]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

## [ TODO ]
# 1) Extract features of all messages into `feat1` with `CountVectorizer` using CHARACTER trigrams
# 2) Repeat with `TfidfVectorizer` and save generated features as `feat2`
print("Extracting features with `CountVectorizer`...")
vectorizer1 = CountVectorizer(analyzer='char', ngram_range=(3, 3))
feat1 = vectorizer1.fit_transform(data['text'])

print("Extracting features with `TfidfVectorizer`...")
vectorizer2 = TfidfVectorizer(analyzer='char', ngram_range=(3, 3))
feat2 = vectorizer2.fit_transform(data['text'])

print("Completed.")

Extracting features with `CountVectorizer`...
Extracting features with `TfidfVectorizer`...
Completed.


We will also try combining two kinds of feature generated above into a single feature. We will use `scipy.sparse.hstack` to concatenate corresonding features for each sample. Remember, concatenated features matrix should have same amount of samples as `feat1` and `feat2`.

In [27]:
from scipy.sparse import hstack

## [ TODO ]
# Concatenate token count features and TF-IDF features on the feature dimension,
# name the combined feature matrix `feat_comb`.
feat_comb = hstack([feat1, feat2])

Finally, let's take a look at the labels. In our dataset, 0 represents messages from real Twitter users and 1 represents spam messages.

In [28]:
y = data["label"].values

## Training

As usual, we will start by doing a 80% / 20% training-test split. Like lab 2, we suggest you to **use the same random seed for all splits** for comparable and reproducible results.

In [29]:
from sklearn.model_selection import train_test_split

## [ TODO ]
# 1) Split token count features into 80/20 training and test sets
# 2) Split TF-IDF features into 80/20 training and test sets
# 3) Split combined features into 80/20 training and test sets

# Token count features
print("Spliting token count features...")
feat1_train, feat1_test, y1_train, y1_test = train_test_split(feat1, y, test_size=0.2, random_state=42)

# TF-IDF features
print("Spliting TF-IDF features...")
feat2_train, feat2_test, y2_train, y2_test = train_test_split(feat2, y, test_size=0.2, random_state=42)

# Combined features
print("Spliting combined features...")
feat_comb_train, feat_comb_test, y_comb_train, y_comb_test = train_test_split(feat_comb, y, test_size=0.2, random_state=42)

print("Completed.")

Spliting token count features...
Spliting TF-IDF features...
Spliting combined features...
Completed.


We are now ready to train **binary classifiers** for spam message detection. Apart from the `LogisticRegression` classifier you have already seen and used during Lab 1, we will try a few more classifiers:

* `MultinomialNB` (Naive Bayes):  
  This classifier is based on the **Naive Bayes** assumption that, *given the class label*, individual features are conditionally independent.  
  For text classification, `MultinomialNB` is used with **token count** or **TF-IDF-like** features.

  It models the probability of a message belonging to a class $ y \in \{\text{spam}, \text{ham}\} $ as:

  $$
  P(y \mid \mathbf{x}) \propto P(y)\prod_{j=1}^{d} P(x_j \mid y)
   $$

  where $\mathbf{x} = (x_1, x_2, \dots, x_d)$ is the feature vector (e.g., word counts), and $P(x_j \mid y)$ is learned from training data.  
  The predicted label is chosen using:

   $$
  y_{pred} = \arg\max_y \; P(y \mid \mathbf{x})
   $$

  In practice, the computation is done in the **log domain** to avoid numerical underflow:

  $$
  y_{pred} = \arg\max_y \left( \log P(y) + \sum_{j=1}^{d} x_j \log P(x_j \mid y) \right)
  $$

* `LinearSVC`:  
  This classifier is based on **linear SVM**, a popular algorithm for binary classification. The linear SVM algorithm treats the feature vector of each sample as a point in a linear space, and tries to find a hyperplane that separates points of the two classes as well as possible.  

  In practice, we optimize a **squared hinge loss with regularization**, since the features may not be perfectly linearly separable:

   $$
  l_{hinge} = \max(0, y(1 - \mathbf{x}^T \mathbf{w}))^2 + \beta ||\mathbf{w}||_2^2
   $$

  Similarly, the classification result is determined by:

  $$
  y_{pred} = sign(1 - \mathbf{x}^T \mathbf{w})
 $$

We have three kinds of features (**token count**, **TF-IDF**, and **combined**) and three kinds of classifiers. Here we will only try three combinations and compare their performance:

* `MultinomialNB` on token count features  
* `LogisticRegression` on TF-IDF features  
  - Try increasing the number of iterations if a warning of non-convergence pops up  
* `LinearSVC` on combined features  
  - For linear SVM, you can ignore the non-convergence warning as it does not affect performance  


In [30]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

## [ TODO ]
# 1) Train a `MultinomialNB` classifier called `model1` on token count training set
# 2) Train a `LogisticRegression` classifier called `model2` on TF-IDF training set
# 3) Train a `LinearSVC` classifier called `model_comb` on combined features training set

print("Training Naive Bayes classifier on token count training set...")
model1 = MultinomialNB()
model1.fit(feat1_train, y1_train)

print("Training logistic regression classifier on TF-IDF training set...")
model2 = LogisticRegression(max_iter=1000) # Increased max_iter to prevent non-convergence warning
model2.fit(feat2_train, y2_train)

print("Training SVM classifier on combined features training set...")
model_comb = LinearSVC(max_iter=1000) # Increased max_iter for robustness, though note says warnings can be ignored
model_comb.fit(feat_comb_train, y_comb_train)

print("Completed.")

Training Naive Bayes classifier on token count training set...
Training logistic regression classifier on TF-IDF training set...
Training SVM classifier on combined features training set...
Completed.


/usr/local/lib/python3.12/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Evaluation

Finally, it's time to evaluate our models. You probably noticed that we have repeated similar code for different feature extraction and classification models. To avoid that, we will use `for` loops to make evaluation simpler:

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, accuracy_score

features = [("Token count", feat1), ("TF-IDF", feat2), ("Combined", feat_comb)]
models = [("Naive Bayes", model1), ("Logistic", model2), ("SVM", model_comb)]

## [ TODO ]
# Complete missing code in the loop body
for i, ((feat_name, feat), (model_name, model)) in enumerate(zip(features, models)):
    # Begin of metrics for combination
    print("====================")
    print(f"{i+1}. {model_name} on {feat_name}:\n")

    # 1) Split features into training and test set
    # (Note: you should use the same random number seed as the one in the training-test split above)
    if feat_name == "Token count":
        X_train, X_test = feat1_train, feat1_test
        y_train, y_test = y1_train, y1_test
    elif feat_name == "TF-IDF":
        X_train, X_test = feat2_train, feat2_test
        y_train, y_test = y2_train, y2_test
    else:  # Combined
        X_train, X_test = feat_comb_train, feat_comb_test
        y_train, y_test = y_comb_train, y_comb_test

    # 2) Predict labels for training data
    pred_train = model.predict(X_train)

    # 3) Print metrics for training set (precision, recall and F1 metrics)
    train_metrics = precision_recall_fscore_support(
        y_train, pred_train, average="weighted"
    )
    print("Training metrics:", train_metrics)
    # Confusion matrix
    print("Training confusion matrix:\n", confusion_matrix(y_train, pred_train), "\n")
    # Accuracy
    print("Train accuracy:", accuracy_score(y_train, pred_train), "\n")

    # 4) Predict labels for test data
    pred_test = model.predict(X_test)

    # 5) Print metrics for test set (precision, recall and F1 metrics)
    test_metrics = precision_recall_fscore_support(
        y_test, pred_test, average="weighted"
    )
    print("Test metrics:", test_metrics)
    # Confusion matrix
    print("Test Confusion Matrix:\n", confusion_matrix(y_test, pred_test), "\n")
    # Accuracy
    print("Test Accuracy:", accuracy_score(y_test, pred_test))

    # End of metrics for combination
    print("====================\n")

1. Naive Bayes on Token count:

Training metrics: (0.9516236107463399, 0.9492, 0.949133912272942, None)
Training confusion matrix:
 [[19694   284]
 [ 1748 18274]] 

Train accuracy: 0.9492 

Test metrics: (0.9485804059535562, 0.9461, 0.9460160173644349, None)
Test Confusion Matrix:
 [[4939   83]
 [ 456 4522]] 

Test Accuracy: 0.9461

2. Logistic on TF-IDF:

Training metrics: (0.9667428167386359, 0.9654, 0.9653761531533964, None)
Training confusion matrix:
 [[19822   156]
 [ 1228 18794]] 

Train accuracy: 0.9654 

Test metrics: (0.9585512986816062, 0.9569, 0.9568551894692657, None)
Test Confusion Matrix:
 [[4957   65]
 [ 366 4612]] 

Test Accuracy: 0.9569

3. SVM on Combined:

Training metrics: (0.9993009797060882, 0.9993, 0.9993000001960001, None)
Training confusion matrix:
 [[19978     0]
 [   28 19994]] 

Train accuracy: 0.9993 

Test metrics: (0.9487999816955784, 0.9488, 0.9487999528950033, None)
Test Confusion Matrix:
 [[4767  255]
 [ 257 4721]] 

Test Accuracy: 0.9488



# Questions
Please answer the following questions. You can just write down the answer below each question in this cell.


1. Please compute `tf-idf` of word-level **bigrams** for the following three messages **by hand**. You should use the TF-IDF definition introduced in class.

  * Message 1: "vegetables are good for health"
  * Message 2: "fruits are good source of vitamins"
  * Message 3: "proteins are good for health"

  Compute and sort intermediate and final results for **all occurring bigrams in the following order**:
  
  ```
  ["vegetables are", "are good", "good for", "for health", "fruits are", "good source", "source of", "of vitamins", "proteins are"]
  ```
Answer:
| Bigram         | M1 | M2 | M3 |
| -------------- | -- | -- | -- |
| vegetables are | y  | n  | n  |
| are good       | y  | y  | y  |
| good for       | y  | n  | y  |
| for health     | y  | n  | y  |
| fruits are     | n  | y  | n  |
| good source    | n  | y  | n  |
| source of      | n  | y  | n  |
| of vitamins    | n  | y  | n  |
| proteins are   | n  | n  | y  |

TF for each message

M1:

| Bigram         | TF         |
| -------------- | ---------- |
| vegetables are | 1/4 = 0.25 |
| are good       | 1/4 = 0.25 |
| good for       | 1/4 = 0.25 |
| for health     | 1/4 = 0.25 |
| fruits are     | 0          |
| good source    | 0          |
| source of      | 0          |
| of vitamins    | 0          |
| proteins are   | 0          |


M2:

| Bigram         | TF        |
| -------------- | --------- |
| vegetables are | 0         |
| are good       | 1/5 = 0.2 |
| good for       | 0         |
| for health     | 0         |
| fruits are     | 1/5 = 0.2 |
| good source    | 1/5 = 0.2 |
| source of      | 1/5 = 0.2 |
| of vitamins    | 1/5 = 0.2 |
| proteins are   | 0         |


M3:
| Bigram         | TF         |
| -------------- | ---------- |
| vegetables are | 0          |
| are good       | 1/4 = 0.25 |
| good for       | 1/4 = 0.25 |
| for health     | 1/4 = 0.25 |
| fruits are     | 0          |
| good source    | 0          |
| source of      | 0          |
| of vitamins    | 0          |
| proteins are   | 1/4 = 0.25 |

DF and IDF
| Bigram         | Appears in (M1,M2,M3) | df | idf = log(3/df)   |
| -------------- | --------------------- | -- | ----------------- |
| vegetables are | M1                    | 1  | log(3/1) ≈ 1.0986 |
| are good       | M1,M2,M3              | 3  | log(3/3) = 0      |
| good for       | M1,M3                 | 2  | log(3/2) ≈ 0.4055 |
| for health     | M1,M3                 | 2  | 0.4055            |
| fruits are     | M2                    | 1  | 1.0986            |
| good source    | M2                    | 1  | 1.0986            |
| source of      | M2                    | 1  | 1.0986            |
| of vitamins    | M2                    | 1  | 1.0986            |
| proteins are   | M3                    | 1  | 1.0986            |


TF - IDF

M1: [0.2747, 0, 0.1014, 0.1014, 0, 0, 0, 0, 0]

M2:
[0,0, 0, 0, 0.2197, 0.2197, 0.2197, 0.2197, 0]

M3:
[0, 0.1014, 0.1014, 0, 0, 0, 0, 0.2747]

-----------------------------------

  
  Then complete the following questions:

  1. Term Frequency (`tf`) of **bigrams for message 2**.

  [0, 0.2, 0, 0, 0.2, 0.2, 0.2, 0.2, 0]

  2. Inverse Document Frequency (`idf`) of **bigrams for message 2**.

[1.0986, 0, 0.4055, 0.4055, 1.0986, 1.0986, 1.0986, 1.0986, 1.0986]

  3. `tf-idf` of **bigrams for message 2**.

[0, 0, 0, 0, 0.2197, 0.2197, 0.2197, 0.2197, 0]

  4. Is the `tf-idf` value for the term "are good" equal to 0? If it's 0, can you explain why it makes sense to have 0 for that term?
  The TF-IDF value for the bigram "are good" is 0. This happens because "are good" appears in all three messages, giving it a document frequency equal to the total number of documents. In the IDF calculation, this results in log(3/3) = log(1) = 0, which when multiplied by the term frequency gives a TF-IDF of 0. Since the bigram occurs in every message, it does not help distinguish one message from another so it has no weight as a discriminative feature.
  5. If we consider unigram (1-gram) based bag of words, how many words in these three messages would be assigned a 0 value tf-idf? And please explain why.
  If we consider a unigram-based bag-of-words representation, two words — "are" and "good" — would have a TF-IDF of 0 across all messages. This is because these words appear in every message, giving them a document frequency equal to the total number of documents. As a result, their IDF is zero, and multiplying by their term frequency gives TF-IDF = 0. Words that appear in all documents carry no information that differentiates one document from another, so TF-IDF appropriately assigns them zero weight.


2. What does `support` mean in the `classification_report`?



In the context of a classification_report, support refers to the number of actual occurrences of each class in the dataset. It represents how many true samples belong to a particular class, regardless of how the model predicted them. Support is important because it provides context for the other metrics like precision, recall, and F1-score; classes with higher support contribute more to the overall performance measures, while classes with very few samples may lead to less reliable metrics. Support tells you the size of each class in the dataset and helps interpret the balance of your data.

3. If the training data is highly unbalanced, is `accuracy` a good metric for the classifiers' performance? Does a high accuracy necessarily means a high f1-score? Compare `accuracy` with `f1-score`. Based on your criterion, which type of classifier is better? Please give some justifications to support your answer.

If the training data is highly unbalanced, accuracy is not a reliable metric, because a classifier could predict only the majority class and still appear highly accurate. A high accuracy does not necessarily imply a high F1-score, since F1 balances precision and recall, reflecting how well the model identifies positive cases. Compared to accuracy, F1-score is more informative for unbalanced datasets. Therefore, a classifier with a higher F1-score is generally better, as it performs well on both majority and minority classes rather than just favoring the majority.

4. For most cases, the amount of bad data is much less than good data. How can you mitigate the data imbalance issue?

One common way to mitigate data imbalance is to resample the dataset. This can be done by oversampling the minority class or undersampling the majority class to reduce its size. Another approach is to use class-weighted algorithms, where misclassifying minority-class samples is penalized more heavily, helping the model pay more attention to them.

## Model Improvements (Optional; Not Included in Homework Score)

As we have pointed out in the first class, machine learning pipeline does not end with a single iteration of training, testing and evaluation. Here, you will repeat these process with different hyper-parameters and try to improve the spam detection model so that it performs even better.

Below are two **potential improvements** that may be beneficial to the performance. You are encouraged to complete either (or both) of them. Your response to these questions are optional and won't be graded, but we will provide feedbacks to your code and analysis if you choose to answer them. You can add code and text cells below the questions as needed.

1. We use character-level trigrams as tokens during the feature extraction stage, however $N = 3$ is a rather arbitrary choice. What will the performance be like if we use shorter N-grams, such as $N = 1$ and $N = 2$? What about longer N-grams, such as $N = 4$? What if we combine different N-grams, like $N = 1, 2, 3$? Modify the feature extraction code above and play with different (range of) $N$ values. What are **the best (range of) $N$ value on the test set of combined features and SVM classifier**? What metrics did you use to reach this conclusion and why do you think this (range of) $N$ value is the best?

2. During the above training and evaluation process, we are only evaluating three combination of feature extractors and classifiers. Can you try to rewrite the training and evaluation code, such that **every combination** of feature extractors and classifiers can be evaluated? What metrics did you use to reach this conclusion and what is the best combination of feature extractor and classifier in terms of test set performance?

## References

1. CRESCI-2017 Dataset: https://botometer.osome.iu.edu/bot-repository/datasets.html
2. TF-IDF: https://en.wikipedia.org/wiki/Tf%E2%80%93idf
3. N-grams: https://en.wikipedia.org/wiki/N-gram
4. `scikit-learn` documentation: https://scikit-learn.org/stable/modules/classes.html
5. Ridge regression: https://en.wikipedia.org/wiki/Tikhonov_regularization
6. SVM: https://en.wikipedia.org/wiki/Support-vector_machine
7. Lasso: https://en.wikipedia.org/wiki/Lasso_(statistics)
8. ElasticNet: https://en.wikipedia.org/wiki/Elastic_net_regularization
9. Kernel method: https://en.wikipedia.org/wiki/Kernel_method